In [2]:
!pip install dash

In [2]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [48]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [49]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [50]:
# Create a dash application
app = dash.Dash(__name__)

In [51]:
# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                 ],
                 value='ALL',
                 placeholder="Select a Launch Site Here",
                 searchable=True
                 ),

    html.Br()
])


In [52]:
# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site
html.Div(dcc.Graph(id='success-pie-chart')),

(Div(Graph(id='success-pie-chart')),)

In [53]:
html.Br(),

(Br(None),)

In [54]:
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                 ],
                 value='ALL',
                 placeholder="Select a Launch Site Here",
                 searchable=True),

    html.Br(),

    html.Div(dcc.Graph(id='success-pie-chart')),

    html.Br(),

    html.P("Payload range (Kg):"),

    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    marks={
                        0: '0',
                        2500: '2500',
                        5000: '5000',
                        7500: '7500',
                        10000: '10000'
                    },
                    value=[min_payload, max_payload]),

    html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])


In [55]:
# TASK 4: Add a scatter chart to show the correlation between payload and launch success
html.Div([
    dcc.Graph(id='success-payload-scatter-chart')
])                       

Div([Graph(id='success-payload-scatter-chart')])

In [56]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        success_counts = spacex_df[spacex_df['class'] == 1]['Launch Site'].value_counts().reset_index()
        success_counts.columns = ['Launch Site', 'Successes']

In [57]:
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        success_counts = spacex_df[spacex_df['class'] == 1]['Launch Site'].value_counts().reset_index()
        success_counts.columns = ['Launch Site', 'Successes']

        fig = px.pie(success_counts,
                     values='Successes',
                     names='Launch Site',
                     title='Total Success Launches by Site')
        return fig
    else:
        site_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        outcome_counts = site_df['class'].value_counts().reset_index()
        outcome_counts.columns = ['Outcome', 'Count']
        outcome_counts['Outcome'] = outcome_counts['Outcome'].replace({1: 'Success', 0: 'Failure'})

        fig = px.pie(outcome_counts,
                     values='Count',
                     names='Outcome',
                     title=f'Total Success vs Failure for site {entered_site}')
        return fig


In [58]:
site_df = pd.DataFrame({
    'class': [1, 0, 1, 1, 0, 1, 0]  
})
outcome_counts = site_df['class'].value_counts().reset_index()
outcome_counts.columns = ['Outcome', 'Count']
outcome_counts['Outcome'] = outcome_counts['Outcome'].replace({1: 'Success', 0: 'Failure'})

In [59]:
def create_pie_chart(outcome_counts, site_name):
    
    fig = px.pie(outcome_counts,
                 values='Count',
                 names='Outcome',
                 title=f'Total Success vs Failure for site {site_name}')
    return fig

In [60]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [
        Input(component_id='site-dropdown', component_property='value'),
        Input(component_id='payload-slider', component_property='value')
    ]
)
def get_scatter_plot(entered_site, payload_range):
    low, high = payload_range

In [62]:
entered_site = 'ALL'  

# Then use it in your code
if entered_site == 'ALL':
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title='Correlation Between Payload and Success for All Sites'
    )
else:
    filtered_site_df = filtered_df[filtered_df['Launch Site'] == entered_site]

In [65]:
low = 1000  
high = 5000  

filtered_df = spacex_df[
    (spacex_df['Payload Mass (kg)'] >= low) &
    (spacex_df['Payload Mass (kg)'] <= high)
]

In [76]:
site_df.columns = site_df.columns.str.strip()


In [79]:
# Print all columns clearly
print("All columns in your dataframe:")
print(site_df.columns.tolist())
print("\n")

# Print the shape to see dimensions
print(f"Dataframe shape: {site_df.shape}")
print("\n")

# Show first few rows
print("First 3 rows:")
print(site_df.head(3))

All columns in your dataframe:
['class']


Dataframe shape: (7, 1)


First 3 rows:
   class
0      1
1      0
2      1


In [78]:
# This will print the exact column names
print(site_df.columns.tolist())

# Find the column that contains 'site' (case-insensitive)
site_column = [col for col in site_df.columns if 'site' in col.lower()][0]
print(f"Using column: '{site_column}'")

entered_site = "Your Site Name"  # Replace with your actual site name

# Use the dynamically found column name
filtered_site_df = site_df[site_df[site_column] == entered_site]

# For the plot columns, let's also check those
print("\nVerifying plot columns exist:")
print(f"'Payload Mass (kg)' exists: {'Payload Mass (kg)' in site_df.columns}")
print(f"'class' exists: {'class' in site_df.columns}")
print(f"'Booster Version Category' exists: {'Booster Version Category' in site_df.columns}")

fig = px.scatter(
    filtered_site_df,
    x='Payload Mass (kg)',
    y='class',
    color='Booster Version Category',
    title=f'Correlation Between Payload and Success for Site: {entered_site}'
)
fig.show()

['class']


IndexError: list index out of range